In [ ]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit, substring, avg, sum

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/gold/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9e3563c7-b18f-4eb2-862f-31b3e20897c7;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 5024ms :: artifacts dl 23ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-ru

In [15]:
# apagar tudo do hdfs

#s_path = "/projeto/fisico/gold/"
b_path = "/projeto/fisico/gold/physical/"

#client.delete(s_path,True)
client.delete(b_path,True)

True

In [4]:
# read data from the silver tables
s_df = spark.table("silver.stats")
b_df = spark.table("silver.bios")

In [ ]:
s_df.printSchema()
b_df.printSchema()

In [5]:
# join to make a flat table
gd_df = s_df \
    .join(b_df, s_df.Player == b_df.name) \
    .select("Player", "height_cm", "weight_kg", "imc")

In [16]:
gd_df.printSchema()
gd_df.toPandas()

root
 |-- Player: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- height_cm: double (nullable = true)
 |-- weight_kg: double (nullable = true)
 |-- imc: double (nullable = true)



,Player,Age,height_cm,weight_kg,imc
0,Caron Butler,34.0,200.66,103.419060,25.682423
1,Lance Thomas,26.0,203.20,106.594207,25.813281
2,Jon Brockman,23.0,200.66,115.666054,28.723762
3,Rudy Fernandez,25.0,198.12,83.914588,21.376561
4,Paul Pierce,33.0,200.66,106.594207,26.470918
...,...,...,...,...,...
19388,Zeke Zawoluk,22.0,200.66,97.522360,24.218074
19389,Fred Scolari,28.0,177.80,81.646627,25.824490
19390,Jack Coleman,29.0,200.66,88.450512,21.965230
19391,Jack Turner,22.0,194.31,83.914588,22.202369


In [7]:
# aggregate data per country, year and product (name and color)
gd_df = gd_df \
    .groupBy("Player") \
    .agg(
        avg(gd_df.height_cm).alias("height_cm"),
        avg(gd_df.weight_kg).alias("weight_kg"),
        avg(gd_df.imc).alias("imc"),
    )

In [9]:
import pyspark.sql.functions as func
gd_df = gd_df \
    .withColumn("height_cm", func.round(gd_df["height_cm"], 2)) \
    .withColumn("weight_kg", func.round(gd_df["weight_kg"], 2)) \
    .withColumn("imc", func.round(gd_df["imc"], 2))
gd_df.show()

+----------------+---------+---------+-----+
|          Player|height_cm|weight_kg|  imc|
+----------------+---------+---------+-----+
| Al-Farouq Aminu|   205.74|    99.79|23.57|
|   Greg Stiemsma|   210.82|   117.93|26.53|
|  Rodney Stuckey|   195.58|    95.25| 24.9|
|  Brandon Ingram|   205.74|    86.18|20.36|
|     Terry Mills|   208.28|   104.33|24.05|
|  Nate Blackwell|   193.04|    77.11|20.69|
|Chuckie Williams|    190.5|    81.65| 22.5|
|     Greg Howard|   205.74|    97.52|23.04|
|        Ed Mikan|    203.2|   104.33|25.26|
|       Red Rocha|   205.74|    83.91|19.82|
|  Svi Mykhailiuk|    203.2|    92.99|22.52|
|   Nassir Little|   198.12|    99.79|25.42|
|       Bob Allen|   205.74|    92.99|21.97|
|  Reggie Harding|   213.36|   112.94|24.81|
|      Al Bianchi|    190.5|    83.91|23.12|
|     John Barber|   198.12|    95.25|24.27|
|     Zach LaVine|   195.58|    91.47|23.91|
|    Andre Miller|    190.5|    90.72| 25.0|
|  Kevin Seraphin|   205.74|   129.27|30.54|
|  Viktor 

In [10]:
# write to delta table
gd_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/gold/physical/")

In [14]:
# check the results in the table
spark.table("gold.physical").show()
spark.table("gold.physical").count()

+----------------+---+------------------+------------------+------------------+
|          Player|Age|         height_cm|         weight_kg|               imc|
+----------------+---+------------------+------------------+------------------+
|      Boris Diaw| 32|             203.2|       113.3980925|        27.4609375|
|   Iman Shumpert| 24|            195.58| 99.79032140000002|26.085343228200372|
|     Jason Smith| 28|            213.36|111.13013065000001|24.409722222222225|
|  Gerald Wallace| 32|            200.66|       97.52235955|24.218074026598305|
|  Craig Brackins| 23|            208.28|104.32624510000001| 24.04669839381321|
|     James Jones| 30|             203.2|       98.88313666|23.945937500000003|
|    Andre Miller| 34|             190.5|         90.718474|24.995555555555555|
| Jermaine O'Neal| 32|            210.82|102.51187562000001| 23.06256350704021|
|     Brent Barry| 36|            198.12| 83.91458845000001| 21.37656147271532|
|    Devin Harris| 24|             190.5

19393